In [4]:
def create_database():
    """データベースとテーブルを作成（既に存在すればリセット）"""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # テーブル作成
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        language TEXT,
        stars INTEGER
    )
    ''')
    
    # 課題用に毎回クリーンな状態にするため既存データを削除
    cursor.execute('DELETE FROM repositories')
    
    conn.commit()
    conn.close()

def parse_stars(star_text):
    """スター数（例: 1.5k）を数値に変換"""
    star_text = star_text.strip()
    if 'k' in star_text:
        return int(float(star_text.replace('k', '')) * 1000)
    try:
        return int(star_text.replace(',', ''))
    except ValueError:
        return 0

# --- メイン処理 ---

# 1. DB初期化
create_database()

# 2. スクレイピング設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

print(f"アクセス中: {TARGET_URL}")
response = requests.get(TARGET_URL, headers=headers)

# ★重要: サーバー負荷軽減のための待機
time.sleep(1)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # リポジトリリストの取得（クラス名は変わる可能性があるため itemprop で指定）
    repo_list = soup.find_all('li', itemprop='owns')
    
    data_to_insert = []
    
    for repo in repo_list:
        # 名前
        name_tag = repo.find('a', itemprop='name codeRepository')
        repo_name = name_tag.get_text(strip=True) if name_tag else "Unknown"
        
        # 言語
        lang_tag = repo.find('span', itemprop='programmingLanguage')
        language = lang_tag.get_text(strip=True) if lang_tag else "None"
        
        # スター数 (リンク先が /stargazers で終わるものを探す)
        star_tag = repo.find('a', href=re.compile(r'/stargazers$'))
        raw_stars = star_tag.get_text(strip=True) if star_tag else "0"
        stars = parse_stars(raw_stars)
        
        data_to_insert.append((repo_name, language, stars))
    
    # DBへ保存
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.executemany('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)', data_to_insert)
    conn.commit()
    conn.close()
    
    print(f"成功: {len(data_to_insert)} 件のデータをデータベース({DB_NAME})に保存しました。")

else:
    print(f"エラー: アクセスに失敗しました (Status: {response.status_code})")

アクセス中: https://github.com/orgs/google/repositories?type=source&sort=stargazers
成功: 0 件のデータをデータベース(github_data.db)に保存しました。
